[![Fixel Algorithms](https://i.imgur.com/AqKHVZ0.png)](https://fixelalgorithms.gitlab.io/)

# AI Program

## Machine Learning - Supervised Learning - Classification Performance Scores / Metrics: Precision, Recall, ROC and AUC - Exercise

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 1.0.000 | 15/03/2024 | Royi Avital | First version                                                      |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/AIProgram/2024_02/0036PerformanceScoreMetrics.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import make_moons
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import auc, balanced_accuracy_score, confusion_matrix, precision_recall_fscore_support, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

# Image Processing

# Machine Learning

# Miscellaneous
import math
import os
from platform import python_version
import random
import timeit

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image
from IPython.display import display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout, SelectionSlider
from ipywidgets import interact

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

Code Notations:

```python
someVar    = 2; #<! Notation for a variable
vVector    = np.random.rand(4) #<! Notation for 1D array
mMatrix    = np.random.rand(4, 3) #<! Notation for 2D array
tTensor    = np.random.rand(4, 3, 2, 3) #<! Notation for nD array (Tensor)
tuTuple    = (1, 2, 3) #<! Notation for a tuple
lList      = [1, 2, 3] #<! Notation for a list
dDict      = {1: 3, 2: 2, 3: 1} #<! Notation for a dictionary
oObj       = MyClass() #<! Notation for an object
dfData     = pd.DataFrame() #<! Notation for a data frame
dsData     = pd.Series() #<! Notation for a series
hObj       = plt.Axes() #<! Notation for an object / handler / function handler
```

### Code Exercise

 - Single line fill

 ```python
 vallToFill = ???
 ```

 - Multi Line to Fill (At least one)

 ```python
 # You need to start writing
 ????
 ```

 - Section to Fill

```python
#===========================Fill This===========================#
# 1. Explanation about what to do.
# !! Remarks to follow / take under consideration.
mX = ???

???
#===============================================================#
```

In [ ]:
# Configuration
# %matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# Matplotlib default color palette
lMatPltLibclr = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2

In [ ]:
# Courses Packages

from DataVisualization import PlotBinaryClassData, PlotConfusionMatrix, PlotLabelsHistogram


In [ ]:
# General Auxiliary Functions

def PlotDecisionBoundaryClosure( numGridPts: int, gridXMin: float, gridXMax: float, gridYMin: float, gridYMax: float, numDigits: int = 1 ) -> Callable:

    # v0       = np.linspace(gridXMin, gridXMax, numGridPts)
    # v1       = np.linspace(gridYMin, gridYMax, numGridPts)
    roundFctr = 10 ** numDigits
    
    # For equal axis
    minVal = np.floor(roundFctr * min(gridXMin, gridYMin)) / roundFctr
    maxVal = np.ceil(roundFctr * max(gridXMax, gridYMax)) / roundFctr
    v0     = np.linspace(minVal, maxVal, numGridPts)
    v1     = np.linspace(minVal, maxVal, numGridPts)
    
    XX0, XX1 = np.meshgrid(v0, v1)
    XX       = np.c_[XX0.ravel(), XX1.ravel()]

    def PlotDecisionBoundary(hDecFun: Callable, hA: plt.Axes = None) -> plt.Axes:
        
        if hA is None:
            hF, hA = plt.subplots(figsize = (8, 6))

        Z = hDecFun(XX)
        Z = Z.reshape(XX0.shape)
            
        hA.contourf(XX0, XX1, Z, colors = CLASS_COLOR, alpha = 0.3, levels = [-0.5, 0.5, 1.5])

        return hA

    return PlotDecisionBoundary



## Exercise - Calibrating the Model Performance

In this exercise we'll learn few approaches dealing with imbalanced data and tuning performance:

 - Resampling.
 - Weighing (Class / Samples).
 - Probability Threshold.

We'll do that using the SVM model, though they generalize to most models.

* <font color='brown'>(**#**)</font> Pay attention that in order to have the probability per class on the _SVC_ class we need to set `probability = True`.
* <font color='brown'>(**#**)</font> The process of `probability = True` is not always consistent with the `decision_function()` method. Hence it is better to use it in the case of the `SVC`.
* <font color='brown'>(**#**)</font> In the above, all approaches are during the training time. One could also search for the best model, score wise, using _Cross Validation_.


In [ ]:
# Parameters

# Data Generation
numSamples0 = 950
numSamples1 = 50

noiseLevel = 0.1

# Test / Train Loop
testSize = 0.5

# Model
paramC      = 1
kernelType  = 'linear'

# Data Visualization
numGridPts = 250


## Generate / Load Data


In [ ]:
# Load Data 

mX, vY = make_moons(n_samples = (numSamples0, numSamples1), noise = noiseLevel)

print(f'The features data shape: {mX.shape}')
print(f'The labels data shape: {vY.shape}')
print(f'The unique values of the labels: {np.unique(vY)}')

### Plot Data

In [ ]:
# Plot the Data

# Class Indices
vIdx0 = vY == 0
vIdx1 = vY == 1

hA = PlotBinaryClassData(mX, vY, axisTitle = 'Samples Data')

### Distribution of Labels

When dealing with classification, it is important to know the balance between the labels within the data set.

In [ ]:
# Distribution of Labels

hA = PlotLabelsHistogram(vY)
plt.show()

## Train SVM Classifier

In [ ]:
# SVM Linear Model

#===========================Fill This===========================#
# 1. Train a model and set the parameter `probability` to `True`
oSVM  = SVC(C = paramC, kernel = kernelType, probability = True).fit(mX, vY) #<! We can do the training in a one liner
#===============================================================#

modelScore = oSVM.score(mX, vY)

print(f'The model score (Accuracy) on the data: {modelScore:0.2%}') #<! Accuracy

### Plot Decision Boundary

We'll display, the linear, decision boundary of the classifier.

In [ ]:
# Decision Boundary Plotter (Per Data!)
# Look at the implementation for an example for a Closure in Python.

PlotDecisionBoundary = PlotDecisionBoundaryClosure(numGridPts, mX[:, 0].min(), mX[:, 0].max(), mX[:, 1].min(), mX[:, 1].max())


In [ ]:
# Decision Boundary
# Plotting the decision boundary.
hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
hA = PlotDecisionBoundary(oSVM.predict, hA)
hA = PlotBinaryClassData(mX, vY, hA = hA, axisTitle = 'Classifier Decision Boundary')
plt.show()

In [ ]:
# Prediction Confidence Level (Probability)

#===========================Fill This===========================#
# 1. Evaluate the decision function for `mX`.
# 2. Calculate the probability function for `mX`.
# !! You should use the `decision_function()` and `predict_proba()` methods.
vD = oSVM.decision_function(mX) #<! Apply the decision function of the data set
mP = oSVM.predict_proba(mX) #<! Probabilities per class
#===============================================================#


* <font color='red'>(**?**)</font> Describe the decision score of the points.
* <font color='red'>(**?**)</font> What are the units of `vD` and `mP`? Why do they have different shapes?

In [ ]:
# Plot the Decision Function Output vs. the Probability
# The built in probability doesn't match the decision function of the classifier!

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
vSampleIdx = list(range(1, mX.shape[0] + 1))

hA.scatter(vSampleIdx, vD > 0, s = 3 * ELM_SIZE_DEF, label = 'Class by Decision Function')
hA.scatter(vSampleIdx, np.argmax(mP, axis = 1), s = ELM_SIZE_DEF, label = 'Class by Probability')
hA.set_xlabel('Sample Index')
hA.set_ylabel('Predicted Class')
hA.legend()

plt.show()

* <font color='red'>(**?**)</font> Explain the graph. Make sure you understand the calculation.
* <font color='red'>(**?**)</font> Which one matches the trained model?

### Alternative Probability Function

The `SVC` class uses the [_Platt Scaling_](https://en.wikipedia.org/wiki/Platt_scaling) for estimating the probabilities.  
As such, it doesn't always match the results given by the _decision boundary_ (Though it is based on it).

In this section an alternative method is presented where:

$$ p \left( \hat{y}_{i} = 1 \mid {d}_{i} \right) = 0.5 \left( 1 + \operatorname{sign} \left( {d}_{i} \right) \left( 1 - {e}^{- \left| {d}_{i} \right|} \right) \right) $$

Where ${d}_{i} = \boldsymbol{w}^{T} \boldsymbol{x}_{i} - b$ is teh "distance" of the point (With a sign) form teh decision boundary.

* <font color='brown'>(**#**)</font> The motivation of this function is giving intuition and not being a calibration process of a function.

* <font color='red'>(**?**)</font> What is required for ${d}_{i}$ to be the actual distance?
* <font color='red'>(**?**)</font> In binary classification, what would be $p \left( \hat{y}_{i} = 0 \mid {d}_{i} \right)$? 

In [ ]:
# Probability function for Binary SVM Classifier
# Idea is to create function with asymptotic behavior
# The input is the per sample output of `decision_function()` method

def SvcBinProb( vD: np.ndarray ) -> np.ndarray:
    mP = np.zeros(shape = (vD.shape[0], 2))

    mP[:, 1] = 0.5 * (1 + np.sign(vD) * (1 - np.exp(-np.abs(vD)))) #>! The probability of the positive class
    mP[:, 0] = 1 - mP[:, 1]

    return mP